# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [27]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
from holoviews import dim
from bokeh.models import HoverTool


# Import API key
from api_keys import geoapify_key

In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,51.82,79,79,10.00,NZ,1692749472
1,1,ondjiva,-17.0667,15.7333,64.11,18,2,6.85,AO,1692749472
2,2,iqaluit,63.7506,-68.5145,49.69,76,100,23.00,CA,1692749473
3,3,port mathurin,-19.6833,63.4167,73.53,76,3,22.03,MU,1692749473
4,4,ilebo,-4.3167,20.5833,73.62,69,16,1.41,CD,1692749473


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
# Define a function to create a map with point size based on humidity
def create_humidity_map(data):
    return gv.Points(data, ['Lng', 'Lat'], 'Humidity').opts(
        color='blue', size=dim('Humidity')/10, cmap='cool', tools=['hover'], 
        title='City Humidity Map', ylabel='Latitude', xlabel='Longitude'
    )

In [30]:
%%capture --no-display
# Create the map using the function
humidity_map = create_humidity_map(city_data_df)

# Display the map
humidity_map



:Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [31]:
# Narrow down cities that fit criteria
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] > 70) & 
    (city_data_df["Max Temp"] < 80) & 
    (city_data_df["Wind Speed"] < 10) & 
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
106,106,sefrou,33.8319,-4.8280,79.75,36,0,2.30,MA,1692749496
117,117,brigantine,39.4101,-74.3646,73.99,69,0,4.61,US,1692749261
145,145,constantia,44.1833,28.6500,77.70,64,0,6.13,RO,1692749502
198,198,baikonur,45.6167,63.3167,78.28,34,0,4.79,KZ,1692749512
382,382,cedro,-6.6067,-39.0622,79.03,45,0,4.25,BR,1692749556


### Step 3: Create a new DataFrame called `hotel_df`.

In [32]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column "Hotel Name" to the DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
106,sefrou,MA,33.8319,-4.8280,36,
117,brigantine,US,39.4101,-74.3646,69,
145,constantia,RO,44.1833,28.6500,64,
198,baikonur,KZ,45.6167,63.3167,34,
382,cedro,BR,-6.6067,-39.0622,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters
params = {
    "type": "hotel",
    "radius": radius,
    "limit": 1,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lng},{lat}"
    params["bias"] = f"circle:{radius}@{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data

Starting hotel search
sefrou - nearest hotel: No hotel found
brigantine - nearest hotel: No hotel found
constantia - nearest hotel: No hotel found
baikonur - nearest hotel: No hotel found
cedro - nearest hotel: No hotel found
selebi-phikwe - nearest hotel: No hotel found
ozieri - nearest hotel: No hotel found
vallo della lucania - nearest hotel: No hotel found
suez - nearest hotel: No hotel found
peruibe - nearest hotel: No hotel found
zakynthos - nearest hotel: No hotel found
ain taya - nearest hotel: No hotel found
saint croix - nearest hotel: No hotel found
sitia - nearest hotel: No hotel found
ibshaway - nearest hotel: No hotel found
chiredzi - nearest hotel: No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
%%capture --no-display

city_data_df = pd.read_csv("output_data/cities.csv")

# Configure the hover tool
hover = HoverTool(
    tooltips=[
        ("City", "@City"),
        ("Country", "@Country"),
        ("Hotel Name", "@{Hotel Name}"),
        ("Humidity", "@Humidity"),
    ]
)

# Configure the map plot with hover tool
hotel_map = gv.Points(city_data_df, ['Lng', 'Lat'], ['Humidity', 'City', 'Country', 'Hotel Name']).opts(
    color='blue', size=dim('Humidity')/10, cmap='cool', tools=[hover],
    title='City Humidity Map with Hotel Information', ylabel='Latitude', xlabel='Longitude'
)

# Display the map
hotel_map


DataError: Supplied data does not contain specified dimensions, the following dimensions were not found: ['Hotel Name']

PandasInterface expects tabular data, for more information on supported datatypes see http://holoviews.org/user_guide/Tabular_Datasets.html